In [249]:
#reddit account
#username:IES_Python
#email:95346725@fsv.cuni.cz

In [1]:
#import sys
#!{sys.executable} -m pip install praw
#!{sys.executable} -m pip install pandas-datareader
from bs4 import BeautifulSoup
import requests
import time
from datetime import date
import pandas as pd
import numpy as np

In [106]:
class Yahoo_Extractor:
    def __init__(self):
        pass

    def getData(self, acro):
        link = 'https://finance.yahoo.com/quote/' + acro + '/history?p=' + acro
        r = requests.get(link)
        r.encoding = 'utf-8-sig'
        self.soup = BeautifulSoup(r.text, 'html.parser')
        lst = []
        stocksDF = pd.DataFrame({'Date':pd.Series([], dtype='str'),
                               acro:pd.Series([], dtype='str')})
        for tr in self.soup.findAll('tr', {'class':'BdT Bdc($seperatorColor) Ta(end) Fz(s) Whs(nw)'}):
            Date = tr.contents[0].text
            try:
                Volume = tr.contents[6].text.replace(",","")
            except IndexError:
                Volume = 'NA'
            stocks_row = pd.DataFrame({'Date':pd.Series([Date], dtype='str'),
                                     acro:pd.Series([Volume], dtype='str')})
            stocksDF = stocksDF.append(stocks_row)
        return stocksDF
    
    def joinData(self, acro):
        finalDF = pd.DataFrame({'Date':pd.Series([], dtype='str')})
        for a in acro:
            stocksDF = self.getData(a)
            finalDF = finalDF.merge(stocksDF, how = 'outer', on = 'Date')
        finalDF = finalDF.drop_duplicates(subset = 'Date') #removing duplicate entries   
        return finalDF
        
        

In [107]:
Y = Yahoo_Extractor()

data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
acro = data[0]
acro = acro['Symbol'].tolist()
#acro = ['MMM', 'ABT', 'ABBV', 'ABMD'] #several acronyms for test

start = time.time()
finalDF = Y.joinData(acro) #the actual procedure
end = time.time()
print(end-start)

3.6294474601745605


In [108]:
today = date.today()
stringEpoch = today.strftime("%d_%m_%y")
s = ''
nameVars = ["C:\\Users\\hso20\\Python\\Project\\YahooData\\",stringEpoch,".csv"]
finalDF.to_csv(path_or_buf=s.join(nameVars),sep = ';', index=True, encoding='utf-8-sig')